In [1]:
from pystac_client import Client
import fsspec
import xarray as xr
import h5py

Generate your token with iam_token.py, please don't share :) 

In [8]:
token = '' 

#### Navigate the catalog and filter

In [2]:
catalog_url = 'https://catalog.maap.eo.esa.int/catalogue/'
catalog = Client.open(catalog_url)

If you are curious about more queryables / filtering options:
`curl -X GET -G https://catalog.maap.eo.esa.int/catalogue/collections/EarthCAREL2InstChecked_MAAP/queryables`
or visit: https://catalog.maap.eo.esa.int/doc/stac.html#granule-search 

In [3]:
search = catalog.search(
    collections=["EarthCAREL2InstChecked_MAAP"], # Currently MAAP has access to this L2 collection 

    filter="productType = 'ATL_ICE_2A' ", # Filter by product type


    #datetime=['2025-04-01T12:47:51.000Z', '2025-04-05T12:47:51.000Z'], # filter by temporal extent 
    
    bbox = [-8, 42, 5, 47] or [9, 43, 23, 47], # combine multiple bbox by OR 
    # bounding box is defined by the bottom left corner (longmin latmin) and the top right corner coordinates (longmax latmax) 

    max_items=10  # Adjust as needed
)

items = list(search.items())
results = dict() # or just {} 
id_lst = []
href_lst = []
for item in items:
    id_lst.append(item.id)
    href_lst.append(item.assets["enclosure_1"].href)

for i, j in zip(id_lst, href_lst):
    results.update({i: j})


In [4]:
print(len(items))

10


In [9]:
fs = fsspec.filesystem("https", headers={"Authorization": f"Bearer {token}"})
href = href_lst[0]
with fs.open(href, "rb") as f:
    ds = xr.open_dataset(f, engine="h5netcdf", group="ScienceData")


In [10]:
ds

<xarray.Dataset> Size: 40MB
Dimensions:                                                 (along_track: 5140,
                                                             JSG_height: 242)
Dimensions without coordinates: along_track, JSG_height
Data variables: (12/18)
    medium_resolution_number_of_joint_standard_grid_pixels  float32 4B ...
    low_resolution_number_of_joint_standard_grid_pixels     float32 4B ...
    joint_standard_grid_index                               (along_track) float64 41kB ...
    latitude                                                (along_track) float64 41kB ...
    longitude                                               (along_track) float64 41kB ...
    time                                                    (along_track) datetime64[ns] 41kB ...
    ...                                                      ...
    ice_water_content                                       (along_track, JSG_height) float32 5MB ...
    ice_water_content_error                                 (along_track, JSG_height) float32 5MB ...
    ice_effective_radius                                    (along_track, JSG_height) float32 5MB ...
    ice_effective_radius_error                              (along_track, JSG_height) float32 5MB ...
    extended_data_quality_status                            (along_track, JSG_height) float32 5MB ...
    quality_status                                          (along_track, JSG_height) float32 5MB ...